In [14]:
import pandas as pd
import gc
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from scipy.stats import pearsonr
import xgboost
import lightgbm

seed_state = 2022


In [23]:
train_X = pd.read_csv("../train_bin.csv")
test_X = pd.read_csv("../test_bin.csv")

In [24]:
train_y = train_X['isDefault']
train_X.drop(labels = ['isDefault'], axis = 1, inplace = True)

In [25]:
drop_list = ['ficoRangeHigh','n3','policyCode']
train_X.drop(labels = drop_list, axis = 1, inplace = True)
test_X.drop(labels = drop_list, axis = 1, inplace = True)

In [26]:
train_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 45 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       800000 non-null  int64  
 1   loanAmnt                 800000 non-null  float64
 2   term                     800000 non-null  int64  
 3   interestRate             800000 non-null  float64
 4   installment              800000 non-null  float64
 5   grade                    800000 non-null  int64  
 6   subGrade                 800000 non-null  int64  
 7   employmentTitle          800000 non-null  float64
 8   employmentLength         800000 non-null  int64  
 9   homeOwnership            800000 non-null  int64  
 10  annualIncome             800000 non-null  int64  
 11  verificationStatus       800000 non-null  int64  
 12  purpose                  800000 non-null  int64  
 13  postCode                 800000 non-null  float64
 14  regi

## 1. 重要特征——暴力组合

In [7]:
"""暴力 组合重要特则"""
importance_feature_list = ['grade','subGrade','term','homeOwnership'
,'issueDate_year','n11','n2','employmentLength','ficoRangeLow','n14',
'loanAmnt','dti','n9','issueDate_month','annualIncome','verificationStatus',
'revolBal',
]
temp_list = [] #暂存组合的特征名
result = []
index = 0
temp_df = pd.DataFrame()
for i in importance_feature_list:
    for j in train_X.columns:
        col = i + '_' + j
        if j + '_' + i in temp_list: #避免重复
            print('重复了')
            continue
        temp_list.append(col)
        corr =  pearsonr(train_X[i] * train_X[j],train_y)
        if corr[1] <= 0.05 and corr[0] > 0.19:
            index += 1
            print(index,col,'\n')

            result.append([col,corr[0],corr[1]]) #存入列表
            temp_df[col] = train_X[i] * train_X[j]
        del col, corr
        gc.collect()
pd.DataFrame(data = result).to_csv("./feature_cross_result.csv")

1 grade_loanAmnt 

2 grade_term 

3 grade_interestRate 

4 grade_installment 

5 grade_grade 

6 grade_subGrade 

7 grade_verificationStatus 

8 grade_dti 

9 grade_ficoRangeLow 

10 grade_openAcc 

11 grade_revolUtil 

12 grade_totalAcc 

13 grade_n1 

14 grade_n2 



2022-10-25 16:03:23,929 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)


15 grade_n4 



2022-10-25 16:03:24,816 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2022-10-25 16:03:25,519 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2022-10-25 16:03:26,211 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)


16 grade_n7 



...

2022-10-25 16:11:18,191 - distributed.utils_perf - WARNING - full garbage collections took 38% CPU time recently (threshold: 10%)


重复了


2022-10-25 16:11:18,948 - distributed.utils_perf - WARNING - full garbage collections took 37% CPU time recently (threshold: 10%)
2022-10-25 16:11:19,781 - distributed.utils_perf - WARNING - full garbage collections took 37% CPU time recently (threshold: 10%)


重复了
重复了
重复了


2022-10-25 16:11:20,569 - distributed.utils_perf - WARNING - full garbage collections took 37% CPU time recently (threshold: 10%)
2022-10-25 16:11:21,319 - distributed.utils_perf - WARNING - full garbage collections took 37% CPU time recently (threshold: 10%)
Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [11]:
train_X = pd.concat([train_X, temp_df], axis = 1)

In [12]:
del temp_df
gc.collect()

2022-10-25 16:14:46,118 - distributed.utils_perf - WARNING - full garbage collections took 34% CPU time recently (threshold: 10%)


3452

In [13]:
train_X.info()

<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 732996 entries, 0 to 732995
Data columns (total 91 columns):
 #   Column                            Non-Null Count   Dtype  
---  --------------------------------  ---------------  -----  
 0   id                                732996 non-null  int64
 1   loanAmnt                          732996 non-null  float64
 2   term                              732996 non-null  int64
 3   interestRate                      732996 non-null  float64
 4   installment                       732996 non-null  float64
 5   grade                             732996 non-null  int64
 6   subGrade                          732996 non-null  int64
 7   employmentTitle                   732996 non-null  float64
 8   employmentLength                  732996 non-null  int64
 9   homeOwnership                     732996 non-null  int64
 10  annualIncome                      732996 non-null  float64
 11  verificationStatus                732996 non-null  int64
 1

In [46]:
train_X['loanAmnt_annualIncome'] = train_X['loanAmnt'] / train_X['annualIncome']
test_df['loanAmnt_annualIncome'] = test_df['loanAmnt'] / test_df['annualIncome']
train_X['loanAmnt_annualIncome'] = train_X['loanAmnt_annualIncome'].apply(lambda x: 0 if x == np.inf else x)
test_X['loanAmnt_annualIncome'] = test_X['loanAmnt_annualIncome'].apply(lambda x: 0 if x == np.inf else x)

train_X['installment_loanAmnt'] = train_X['installment'] / train_X['loanAmnt']
test_df['installment_loanAmnt'] = test_df['installment'] / test_df['loanAmnt']

train_X['issueDate_year_earliesCreditLine_year'] = train_X['issueDate_year'] - train_X['earliesCreditLine_year']
test_df['issueDate_year_earliesCreditLine_year'] = test_df['issueDate_year'] - test_df['earliesCreditLine_year']

train_X['loanAmnt_totalAcc'] = train_X['loanAmnt'] / train_X['totalAcc']
test_df['loanAmnt_totalAcc'] = test_df['loanAmnt'] / test_df['totalAcc']

train_X['openAcc_totalAcc'] = train_X['openAcc'] / train_X['totalAcc']
test_df['openAcc_totalAcc'] = test_df['openAcc'] / test_df['totalAcc']

train_X['purpose_totalAcc'] = train_X['purpose'] * train_X['totalAcc']
test_df['purpose_totalAcc'] = test_df['purpose'] * test_df['totalAcc']


In [92]:
feature_names = list(train_X.columns)

#包外验证
df_oof = train_X[['id']].copy()
df_oof['label'] = train_y
df_oof['prob'] = 0

#存放测试结果
prediction = test_df[['id']]
prediction['prob'] = 0

#重要特征列表
df_importance_list = []

model = xgboost.XGBClassifier(
                            n_estimators = 10000,
                            n_jobs = -1,
                            max_depth = 6,
                            learning_rate = 0.01,
                            subsample = 0.8,
                            gamma = 0.3,
                            reg_alpha = 3,
                            reg_lambda = 1,
                            random_state = seed_state)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed_state)

for fold_id,(train_idx, val_idx) in enumerate(kfold.split(train_X, train_y)):
    train_X_kfold = train_X.iloc[train_idx]
    train_y_kfold = train_y.iloc[train_idx]
    val_X_kfold = train_X.iloc[val_idx]
    val_y_kfold = train_y.iloc[val_idx]
    
    print('\nFold_{} Training ========================\n'.format(
        fold_id + 1))
    
    xgb_model = model.fit(train_X_kfold,
                          train_y_kfold,
                          eval_set=[(train_X_kfold, train_y_kfold), (val_X_kfold, val_y_kfold)],
                          verbose=50,
                          eval_metric='auc',
                          early_stopping_rounds=50)
    
    pred_val = xgb_model.predict_proba(val_X_kfold)[:, 1]
    df_oof.loc[val_idx, 'prob'] = pred_val
    
    
    pred_test = xgb_model.predict_proba(test_df)[:, 1]
    prediction['prob'] += pred_test / kfold.n_splits

    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': xgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)

    del xgb_model, pred_val, pred_test, train_X_kfold, train_y_kfold, val_X_kfold, val_y_kfold
    gc.collect()

C:\Users\Mr.Li\AppData\Local\Temp\ipykernel_17264\2517155957.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction['prob'] = 0



Fold_1 Training ========================



E:\anaconda\envs\kaggle\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
E:\anaconda\envs\kaggle\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-auc:0.70556	validation_1-auc:0.70383


KeyboardInterrupt: 

In [90]:
model.evals_result

NameError: name 'modelbest_ntree_limit' is not defined

In [59]:
df_importance_list

[                                    column  importance
 0                                       id    0.003147
 1                                 loanAmnt    0.005039
 2                                     term    0.026692
 3                             interestRate    0.004421
 4                              installment    0.006069
 ..                                     ...         ...
 92                      issueDate_year_n14    0.007922
 93          issueDate_year_issueDate_month    0.005823
 94            issueDate_year_issueDate_day    0.000000
 95  issueDate_year_earliesCreditLine_month    0.003463
 96   issueDate_year_earliesCreditLine_year    0.005083
 
 [97 rows x 2 columns],
                                     column  importance
 0                                       id    0.003113
 1                                 loanAmnt    0.005172
 2                                     term    0.024438
 3                             interestRate    0.004154
 4                    

In [60]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].mean().sort_values(ascending = False).reset_index()

In [61]:
df_importance.to_csv("./feature_importance_issueDate_year_others.csv")

In [32]:
pd.read_csv("./untitled.txt").to_csv("./ss.csv",encoding = 'gbk')

In [34]:
prediction.to_csv("./submition.csv")

In [16]:
feature_names = list(train_X.columns)

#包外验证
df_oof = train_X[['id']].copy()
df_oof['label'] = train_y
df_oof['prob'] = 0

#存放测试结果
prediction = test_df[['id']]
prediction['prob'] = 0

#重要特征列表
df_importance_list = []

model = RandomForestClassifier(
                            n_estimators = 900,
                            n_jobs = -1,
                            max_depth = 6,
                    random_state = seed_state)


xgb_model = model.fit(train_X,train_y)



ValueError: No objects to concatenate